In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

In [3]:
date = '2023.09.22'
df = pd.read_pickle(f'/data2/prepared_data/fac_ret_{date}.pkl')
df.dropna(inplace=True)

In [4]:
df.shape

(21252313, 31)

In [5]:
df= df.replace([np.inf, -np.inf], np.nan).dropna(subset=df.columns)
df.shape

(21252313, 31)

In [6]:
X, y = df.drop(columns=df.columns[:5]), df[['1m']]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
print(X_train.shape)
print(y_train.shape)

print(np.isnan(X_train).any().any())
print(np.isnan(y_train).any().any())
print(np.isinf(X_train).any().any())
print(np.isinf(y_train).any().any())
print(np.isfinite(X_train).all().any())
print(np.isfinite(y_train).all().any())

(17001850, 26)
(17001850, 1)
False
False
False
False
True
True


In [28]:


# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train)
dvalid_reg = xgb.DMatrix(X_valid, y_valid)

In [ ]:
y_valid

In [27]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

evals = [(dtrain_reg, "train"), (dvalid_reg, "validation")]
n = 5000
# n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=100,
   early_stopping_rounds=50,
)

ValueError: feature_names mismatch: ['td_ret_v_prod_5min', 'td_p_v_ratio_3s_1min', 'td_p_v_ratio_3s_5min', 'close_ret_15min', 'close_ret_5min', 'close_ret_1min', 'ohlc_rat', 'clh', 'clh_delta', 'td_buy_rank', 'td_sell_rank', 'close_adjusted', 'retail_dir_tsrank_10min', 'retail_dir_rowrank', 'wb1_tsrank', 'wb5_tsrank', 'wb10_tsrank', 'td_v_ratio', 'bs_v1_tsrank', 'bs_v5_tsrank', 'bs_pv1_tsrank', 'bs_pv5_tsrank', 'en_b_price_tsrank', 'en_s_price_tsrank', 'bs_press', 'en_v_order_inbalance'] ['1m']
expected td_v_ratio, close_ret_15min, bs_pv1_tsrank, en_s_price_tsrank, retail_dir_tsrank_10min, en_v_order_inbalance, wb10_tsrank, close_adjusted, wb5_tsrank, td_buy_rank, bs_v1_tsrank, bs_v5_tsrank, td_p_v_ratio_3s_5min, bs_pv5_tsrank, ohlc_rat, en_b_price_tsrank, bs_press, retail_dir_rowrank, close_ret_1min, td_ret_v_prod_5min, wb1_tsrank, clh_delta, td_sell_rank, close_ret_5min, clh, td_p_v_ratio_3s_1min in input data
training data did not have the following fields: 1m

In [26]:
model.feature_names

['td_ret_v_prod_5min',
 'td_p_v_ratio_3s_1min',
 'td_p_v_ratio_3s_5min',
 'close_ret_15min',
 'close_ret_5min',
 'close_ret_1min',
 'ohlc_rat',
 'clh',
 'clh_delta',
 'td_buy_rank',
 'td_sell_rank',
 'close_adjusted',
 'retail_dir_tsrank_10min',
 'retail_dir_rowrank',
 'wb1_tsrank',
 'wb5_tsrank',
 'wb10_tsrank',
 'td_v_ratio',
 'bs_v1_tsrank',
 'bs_v5_tsrank',
 'bs_pv1_tsrank',
 'bs_pv5_tsrank',
 'en_b_price_tsrank',
 'en_s_price_tsrank',
 'bs_press',
 'en_v_order_inbalance']

In [48]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dvalid_reg)

In [31]:
date = '2024.02.19'
df_tt = pd.read_pickle(f'/home/wangzirui/workspace/data/fac_ret_{date}.pkl')


FileNotFoundError: [Errno 2] No such file or directory: '/home/wangzirui/workspace/data/fac_ret_2024.02.19.pkl'

In [3]:
def preprocess(df):
    df= df.replace([np.inf, -np.inf], np.nan).dropna(subset=df.columns)
    
    return df

def check(x:pd.DataFrame, y:pd.DataFrame):
    x = x.loc[:,x.columns[5:]].to_numpy()
    y = y.to_numpy()
    print(x.shape)
    print(np.isnan(x).any().any())
    print(np.isinf(x).any().any())
    print(np.isfinite(x).all().any())
    
    print(y.shape)
    print(np.isnan(y).any().any())
    print(np.isinf(y).any().any())
    print(np.isfinite(y).all().any())
    

In [6]:
df_tt = preprocess(df_tt)

(11695611, 60)
False
False
True
(11695611, 1)
False
False
True


In [7]:
X_test, y_test = df_tt.drop(columns=df_tt.columns[:5]), df_tt[['1m']]

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [30]:
# model.save_model('/home/wangzirui/workspace/models/preliminary_model.json')

model = xgb.Booster()
model.load_model('/home/wangzirui/workspace/models/preliminary_model.json')

In [117]:
check(X_test, y_test)

(11695611, 55)
False
False
True
(11695611, 1)
False
False
True


In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_testScaled = scaler.fit_transform(X_test)

/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/sklearn/utils/extmath.py:1076: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count


In [15]:
X_testScaled = pd.DataFrame(X_testScaled, columns=X_test.columns, index=X_test.index)

In [16]:
dtest_reg = xgb.DMatrix(X_testScaled, y_test)

In [17]:
preds = model.predict(dtest_reg)

In [18]:
y_test.head(5)

,1m
0,0.003106
8,0.010101
12,0.006557
13,-0.005417
14,0.005900


In [19]:
df_test = df_tt.loc[y_test.index.to_list(), df_tt.columns[:5]]

In [29]:
df_test['factor'] = preds

NameError: name 'preds' is not defined

In [21]:
df_test.set_index(['tradetime', 'securityid'], inplace=True)
df_test.index.set_names(['date', 'asset'], inplace=True)

In [22]:
df_test.head()

1m        3m        5m    factor
date                securityid                                        
2024-02-19 09:45:00 000001.SZ   0.003106  0.004141  0.002070  0.020396
                    000010.SZ   0.010101  0.010101  0.005051  0.014754
                    000016.SZ   0.006557  0.013115  0.013115  0.013283
                    000017.SZ  -0.005417 -0.004334  0.001083 -0.006651
                    000019.SZ   0.005900  0.005900  0.008850  0.033655

In [164]:
from alphalens import performance as perf
def get_factor_ic_summary_info(data):
    group_neutral = False
    ic_data = perf.factor_information_coefficient(data, group_neutral)


    ic_summary_table = pd.DataFrame()
    ic_summary_table["IC Mean"] = ic_data.mean()
    ic_summary_table["IC Std."] = ic_data.std()
    ic_summary_table["Risk-Adjusted IC"] = \
        ic_data.mean() / ic_data.std()
    t_stat, p_value = stats.ttest_1samp(ic_data, 0, nan_policy='omit')
    ic_summary_table["t-stat(IC)"] = t_stat
    ic_summary_table["p-value(IC)"] = p_value
    ic_summary_table["IC Skew"] = stats.skew(ic_data, nan_policy='omit')
    ic_summary_table["IC Kurtosis"] = stats.kurtosis(ic_data, nan_policy='omit')
    ic_summary_table['IC win rate'] = (ic_data > 0).sum() / ic_data.count()
    return ic_summary_table

In [165]:
get_factor_ic_summary_info(df_test)

KeyError: 'Level date not found'